In [1]:
import pandas as pd
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-806f725ebc19>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
# Import proximity data
prox = pd.read_csv('proximity_data_aggregated_BC_HSD.csv')

# Replace some HR names to match with other data
prox["HSDA_Name"].replace({"Kootenay Boundary": "Kootenay-Boundary", 
                           "Thompson Cariboo Shuswap":"Thompson/Cariboo"}, inplace=True)

prox.info()
prox.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HSDA_Name                  16 non-null     object 
 1   amenity_dense              16 non-null     float64
 2   prox_idx_emp_HSDwt         16 non-null     float64
 3   prox_idx_pharma_HSDwt      16 non-null     float64
 4   prox_idx_childcare_HSDwt   16 non-null     float64
 5   prox_idx_health_HSDwt      16 non-null     float64
 6   prox_idx_grocery_HSDwt     16 non-null     float64
 7   prox_idx_educpri_HSDwt     16 non-null     float64
 8   prox_idx_educsec_HSDwt     16 non-null     float64
 9   prox_idx_lib_HSDwt         16 non-null     float64
 10  prox_idx_parks_HSDwt       16 non-null     float64
 11  prox_idx_transit_HSDwt     16 non-null     float64
 12  prox_idx_emp_HSDmed        16 non-null     float64
 13  prox_idx_pharma_HSDmed     16 non-null     float64
 

,HSDA_Name,amenity_dense,prox_idx_emp_HSDwt,prox_idx_pharma_HSDwt,prox_idx_childcare_HSDwt,prox_idx_health_HSDwt,prox_idx_grocery_HSDwt,prox_idx_educpri_HSDwt,prox_idx_educsec_HSDwt,prox_idx_lib_HSDwt,prox_idx_parks_HSDwt,prox_idx_transit_HSDwt,prox_idx_emp_HSDmed,prox_idx_pharma_HSDmed,prox_idx_childcare_HSDmed,prox_idx_health_HSDmed,prox_idx_grocery_HSDmed,prox_idx_educpri_HSDmed,prox_idx_educsec_HSDmed,prox_idx_lib_HSDmed,prox_idx_parks_HSDmed,prox_idx_transit_HSDmed,HSDPOP
0,Central Vancouver Island,0.058901,0.002829,0.003438,0.006786,0.001929,0.003677,0.010280,0.015565,0.005572,0.012972,0.000770,0.0045,0.0237,0.0204,0.0020,0.03990,0.05650,0.08460,0.0753,0.04470,0.0015,270229.0
1,East Kootenay,0.029567,0.000409,0.000961,0.000862,0.000545,0.001308,0.003625,0.002900,0.002574,0.002042,0.000138,0.0016,0.0257,0.0163,0.0028,0.04170,0.05105,0.05570,0.0765,0.03030,0.0049,81058.0
2,Fraser East,0.093064,0.002974,0.003076,0.004358,0.001183,0.002797,0.010082,0.004472,0.004084,0.006252,0.000617,0.0101,0.0239,0.0304,0.0037,0.03500,0.06830,0.05730,0.0745,0.03890,0.0027,300097.0
3,Fraser North,0.245326,0.022334,0.016234,0.026787,0.008499,0.023203,0.039969,0.042301,0.012323,0.014429,0.006613,0.0580,0.0342,0.0682,0.0166,0.05945,0.11090,0.11980,0.0693,0.03590,0.0130,648457.0
4,Fraser South,0.157869,0.020259,0.012018,0.022109,0.005515,0.013357,0.040044,0.054237,0.011668,0.033754,0.003548,0.0381,0.0226,0.0434,0.0068,0.03630,0.08110,0.11395,0.0707,0.06005,0.0056,780984.0


In [3]:
# Import HR_UID for each HSD
hsd = pd.read_csv('HSD_HA_names_id.csv')
hsd.info()
hsd.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  16 non-null     int64 
 1   phu_name    16 non-null     object
 2   HR_UID      16 non-null     int64 
 3   HA_Name     16 non-null     object
 4   HSDA_Name   16 non-null     object
 5   peer_group  16 non-null     object
 6   peer_char   16 non-null     object
dtypes: int64(2), object(5)
memory usage: 1.0+ KB


,Unnamed: 0,phu_name,HR_UID,HA_Name,HSDA_Name,peer_group,peer_char
0,0,Central Vancouver Island Health Service Delivery Area,5942,Vancouver Island,Central Vancouver Island,C,Urban-rural mix
1,1,East Kootenay Health Service Delivery Area,5911,Interior,East Kootenay,D,Rural
2,2,Fraser East Health Service Delivery Area,5921,Fraser,Fraser East,B,Moderate urban
3,3,Fraser North Health Service Delivery Area,5922,Fraser,Fraser North,H,Large urban
4,4,Fraser South Health Service Delivery Area,5923,Fraser,Fraser South,H,Large urban


In [4]:
# Merge prox and hsd so that the prox data will have the HR_UID column
prox = pd.merge(left=prox, right=hsd, how='left', left_on='HSDA_Name', right_on='HSDA_Name')
prox.info()
prox.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HSDA_Name                  16 non-null     object 
 1   amenity_dense              16 non-null     float64
 2   prox_idx_emp_HSDwt         16 non-null     float64
 3   prox_idx_pharma_HSDwt      16 non-null     float64
 4   prox_idx_childcare_HSDwt   16 non-null     float64
 5   prox_idx_health_HSDwt      16 non-null     float64
 6   prox_idx_grocery_HSDwt     16 non-null     float64
 7   prox_idx_educpri_HSDwt     16 non-null     float64
 8   prox_idx_educsec_HSDwt     16 non-null     float64
 9   prox_idx_lib_HSDwt         16 non-null     float64
 10  prox_idx_parks_HSDwt       16 non-null     float64
 11  prox_idx_transit_HSDwt     16 non-null     float64
 12  prox_idx_emp_HSDmed        16 non-null     float64
 13  prox_idx_pharma_HSDmed     16 non-null     float64
 

,HSDA_Name,amenity_dense,prox_idx_emp_HSDwt,prox_idx_pharma_HSDwt,prox_idx_childcare_HSDwt,prox_idx_health_HSDwt,prox_idx_grocery_HSDwt,prox_idx_educpri_HSDwt,prox_idx_educsec_HSDwt,prox_idx_lib_HSDwt,prox_idx_parks_HSDwt,prox_idx_transit_HSDwt,prox_idx_emp_HSDmed,prox_idx_pharma_HSDmed,prox_idx_childcare_HSDmed,prox_idx_health_HSDmed,prox_idx_grocery_HSDmed,prox_idx_educpri_HSDmed,prox_idx_educsec_HSDmed,prox_idx_lib_HSDmed,prox_idx_parks_HSDmed,prox_idx_transit_HSDmed,HSDPOP,Unnamed: 0,phu_name,HR_UID,HA_Name,peer_group,peer_char
0,Central Vancouver Island,0.058901,0.002829,0.003438,0.006786,0.001929,0.003677,0.010280,0.015565,0.005572,0.012972,0.000770,0.0045,0.0237,0.0204,0.0020,0.03990,0.05650,0.08460,0.0753,0.04470,0.0015,270229.0,0,Central Vancouver Island Health Service Delivery Area,5942,Vancouver Island,C,Urban-rural mix
1,East Kootenay,0.029567,0.000409,0.000961,0.000862,0.000545,0.001308,0.003625,0.002900,0.002574,0.002042,0.000138,0.0016,0.0257,0.0163,0.0028,0.04170,0.05105,0.05570,0.0765,0.03030,0.0049,81058.0,1,East Kootenay Health Service Delivery Area,5911,Interior,D,Rural
2,Fraser East,0.093064,0.002974,0.003076,0.004358,0.001183,0.002797,0.010082,0.004472,0.004084,0.006252,0.000617,0.0101,0.0239,0.0304,0.0037,0.03500,0.06830,0.05730,0.0745,0.03890,0.0027,300097.0,2,Fraser East Health Service Delivery Area,5921,Fraser,B,Moderate urban
3,Fraser North,0.245326,0.022334,0.016234,0.026787,0.008499,0.023203,0.039969,0.042301,0.012323,0.014429,0.006613,0.0580,0.0342,0.0682,0.0166,0.05945,0.11090,0.11980,0.0693,0.03590,0.0130,648457.0,3,Fraser North Health Service Delivery Area,5922,Fraser,H,Large urban
4,Fraser South,0.157869,0.020259,0.012018,0.022109,0.005515,0.013357,0.040044,0.054237,0.011668,0.033754,0.003548,0.0381,0.0226,0.0434,0.0068,0.03630,0.08110,0.11395,0.0707,0.06005,0.0056,780984.0,4,Fraser South Health Service Delivery Area,5923,Fraser,H,Large urban


In [5]:
# Import health data
comorbid = pd.read_csv('comorbidity_data_percent_BC.csv')
comorbid['HSDA_Name'] = comorbid['phu_name'].str.replace(r' Health Service Delivery Area$', '')
comorbid.info()
comorbid.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 28 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   phu_name                                                                         16 non-null     object 
 1   HR_UID                                                                           16 non-null     int64  
 2   Arthritis (15 years and over)                                                    16 non-null     float64
 3   Asthma                                                                           16 non-null     float64
 4   Body mass index, adjusted self-reported, adult (18 years and over), obese        16 non-null     float64
 5   Body mass index, adjusted self-reported, adult (18 years and over), overweight   16 non-null     float64
 6   Body mass in

,phu_name,HR_UID,Arthritis (15 years and over),Asthma,"Body mass index, adjusted self-reported, adult (18 years and over), obese","Body mass index, adjusted self-reported, adult (18 years and over), overweight","Body mass index, self-reported, youth (12 to 17 years old), overweight or obese",Breast milk feeding initiation,Chronic obstructive pulmonary disease (COPD; 35 years and over),"Current smoker, daily","Current smoker, daily or occasional",Diabetes,"Exclusive breastfeeding, at least 6 months",Has a regular healthcare provider,Heavy drinking,High blood pressure,Influenza immunization in the past 12 months,"Life satisfaction, satisfied or very satisfied",Mood disorder,"Perceived health, fair or poor","Perceived health, very good or excellent","Perceived life stress, most days quite a bit or extremely stressful","Perceived mental health, fair or poor","Perceived mental health, very good or excellent","Physical activity, 150 minutes per week, adult (18 years and over)","Physical activity, average 60 minutes per day, youth (12 to 17 years old)","Sense of belonging to local community, somewhat strong or very strong",HSDA_Name
0,Central Vancouver Island Health Service Delivery Area,5942,25.8,9.3,23.7,39.7,26.8,92.1,6.0,9.1,12.6,5.9,49.1,87.9,19.8,17.8,36.4,92.9,11.6,13.8,56.5,18.6,10.3,66.8,65.3,60.0,72.4,Central Vancouver Island
1,East Kootenay Health Service Delivery Area,5911,24.1,7.8,30.4,35.9,NaN,100.0,4.8,11.9,17.3,8.5,NaN,82.9,24.4,18.6,31.2,91.6,8.0,14.5,61.8,17.3,8.6,67.9,71.9,73.3,76.5,East Kootenay
2,Fraser East Health Service Delivery Area,5921,20.6,10.6,28.1,39.5,17.1,95.1,6.1,9.1,12.1,7.1,44.5,87.4,14.0,19.5,34.8,91.9,10.2,14.3,57.5,22.2,9.6,66.1,61.8,59.4,72.5,Fraser East
3,Fraser North Health Service Delivery Area,5922,15.1,7.2,22.1,33.8,27.0,99.2,2.8,6.3,11.3,5.5,56.3,82.8,16.1,14.9,30.9,91.4,9.0,11.2,58.5,20.4,9.1,64.2,59.3,64.9,65.8,Fraser North
4,Fraser South Health Service Delivery Area,5923,17.2,5.6,26.7,36.3,24.8,97.7,1.9,5.7,8.8,6.2,49.4,83.0,18.0,15.7,34.8,91.2,8.0,10.5,62.1,23.7,6.4,69.3,59.0,61.6,73.2,Fraser South


In [6]:
comorbid['HSDA_Name'].value_counts()

Fraser South                   1
Fraser East                    1
Northeast                      1
North Vancouver Island         1
Thompson/Cariboo               1
Northwest                      1
Vancouver                      1
Richmond                       1
Okanagan                       1
South Vancouver Island         1
Fraser North                   1
Northern Interior              1
North Shore/Coast Garibaldi    1
Central Vancouver Island       1
East Kootenay                  1
Kootenay-Boundary              1
Name: HSDA_Name, dtype: int64

In [7]:
# Import census data
census = pd.read_csv('census_data_BC.csv')
census = census.rename(columns = {'GEO_CODE (POR)':'HR_UID'})
census['HSDA_Name'] = census['GEO_NAME'].str.replace(r' Health Service Delivery Area$', '')
census.info()
census.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 29 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   GEO_NAME                                                         16 non-null     object 
 1   HR_UID                                                           16 non-null     int64  
 2   Population density per square kilometre                          16 non-null     float64
 3   15 to 64 years                                                   16 non-null     float64
 4   65 years and over                                                16 non-null     float64
 5   Median age of the population                                     16 non-null     float64
 6   Average household size                                           16 non-null     float64
 7   Median total income of households in 2015 ($) 

,GEO_NAME,HR_UID,Population density per square kilometre,15 to 64 years,65 years and over,Median age of the population,Average household size,Median total income of households in 2015 ($),Median after-tax income of households in 2015 ($),Single-detached house prop,Married or living common law prop,Not married and not living common law prop,Owner prop,Renter prop,One person or fewer per room prop,More than 1 person per room prop,Suitable prop,Not suitable prop,"No certificate, diploma or degree prop",Secondary (high) school diploma or equivalency certificate prop,"Postsecondary certificate, diploma or degree prop",In the labour force prop,Employed prop,Unemployed prop,Not in the labour force prop,"Car, truck, van - as a driver prop",Non-movers prop,Movers prop,HSDA_Name
0,Central Vancouver Island Health Service Delivery Area,5942,22.2,60.8,25.5,50.3,2.2,62238.0,55443.0,0.686457,0.602353,0.397625,0.747203,0.250212,0.992965,0.006993,0.971519,0.028481,0.168131,0.295599,0.536314,0.561081,0.921212,0.078828,0.438919,0.813119,0.848688,0.151331,Central Vancouver Island
1,East Kootenay Health Service Delivery Area,5911,1.8,63.5,21.0,46.4,2.3,71383.0,62283.0,0.704944,0.636808,0.363192,0.780615,0.218804,0.994051,0.005949,0.979106,0.020894,0.169656,0.295151,0.535193,0.626150,0.921088,0.078791,0.373926,0.790681,0.862341,0.137659,East Kootenay
2,Fraser East Health Service Delivery Area,5921,26.1,63.9,18.0,41.2,2.7,69321.0,61172.0,0.521651,0.600962,0.399018,0.731092,0.266691,0.978300,0.021700,0.952722,0.047278,0.207510,0.339386,0.453146,0.633949,0.932884,0.067116,0.366051,0.844468,0.845311,0.154706,Fraser East
3,Fraser North Health Service Delivery Area,5922,226.4,70.3,14.7,40.9,2.5,73084.0,63682.0,0.308193,0.577093,0.422917,0.679347,0.320633,0.968773,0.031207,0.929728,0.070272,0.128825,0.293797,0.577369,0.663274,0.941662,0.058352,0.336716,0.677461,0.845875,0.154133,Fraser North
4,Fraser South Health Service Delivery Area,5923,937.8,66.9,15.7,40.3,2.9,79638.0,69572.0,0.401202,0.603680,0.396320,0.734602,0.265305,0.956942,0.043058,0.928365,0.071635,0.174533,0.325161,0.500306,0.652876,0.940185,0.059815,0.347124,0.766113,0.850788,0.149212,Fraser South


In [11]:
# Merge prox, comorbid and census data on HRUID
from functools import reduce

merged = reduce(lambda x,y: pd.merge(x,y, on='HR_UID', how='outer'), [census, prox, comorbid])
merged.columns.values

array(['GEO_NAME', 'HR_UID', 'Population density per square kilometre',
       '15 to 64 years', '65 years and over',
       'Median age of the population', 'Average household size',
       'Median total income of households in 2015 ($)',
       'Median after-tax income of households in 2015 ($)',
       'Single-detached house prop', 'Married or living common law prop',
       'Not married and not living common law prop', 'Owner prop',
       'Renter prop', 'One person or fewer per room prop',
       'More than 1 person per room prop', 'Suitable prop',
       'Not suitable prop', 'No certificate, diploma or degree prop',
       'Secondary (high) school diploma or equivalency certificate prop',
       'Postsecondary certificate, diploma or degree prop',
       'In the labour force prop', 'Employed prop', 'Unemployed prop',
       'Not in the labour force prop',
       'Car, truck, van - as a driver prop', 'Non-movers prop',
       'Movers prop', 'HSDA_Name_x', 'HSDA_Name_y', 'amenity_de

In [12]:
# Reorder columns 
# Drop 
merged = merged[['HR_UID', 'HSDA_Name', 'HSDPOP', 'peer_group', 'peer_char','HA_Name', 
                 'Population density per square kilometre',
       '15 to 64 years', '65 years and over',
       'Median age of the population', 'Average household size',
       'Median total income of households in 2015 ($)',
       'Median after-tax income of households in 2015 ($)',
       'Single-detached house prop', 'Married or living common law prop',
       'Not married and not living common law prop', 'Owner prop',
       'Renter prop', 'One person or fewer per room prop',
       'More than 1 person per room prop', 'Suitable prop',
       'Not suitable prop', 'No certificate, diploma or degree prop',
       'Secondary (high) school diploma or equivalency certificate prop',
       'Postsecondary certificate, diploma or degree prop',
       'In the labour force prop', 'Employed prop', 'Unemployed prop',
       'Not in the labour force prop',
       'Car, truck, van - as a driver prop', 'Non-movers prop',
       'Movers prop', 'amenity_dense',
       'prox_idx_emp_HSDwt', 'prox_idx_pharma_HSDwt', 'prox_idx_childcare_HSDwt',
       'prox_idx_health_HSDwt', 'prox_idx_grocery_HSDwt', 'prox_idx_educpri_HSDwt',
       'prox_idx_educsec_HSDwt', 'prox_idx_lib_HSDwt', 'prox_idx_parks_HSDwt',
       'prox_idx_transit_HSDwt', 'prox_idx_emp_HSDmed', 'prox_idx_pharma_HSDmed',
       'prox_idx_childcare_HSDmed', 'prox_idx_health_HSDmed',
       'prox_idx_grocery_HSDmed', 'prox_idx_educpri_HSDmed',
       'prox_idx_educsec_HSDmed', 'prox_idx_lib_HSDmed', 'prox_idx_parks_HSDmed',
       'prox_idx_transit_HSDmed', 'Arthritis (15 years and over)',
       'Asthma',
       'Body mass index, adjusted self-reported, adult (18 years and over), obese',
       'Body mass index, adjusted self-reported, adult (18 years and over), overweight',
       'Body mass index, self-reported, youth (12 to 17 years old), overweight or obese',
       'Breast milk feeding initiation',
       'Chronic obstructive pulmonary disease (COPD; 35 years and over)',
       'Current smoker, daily', 'Current smoker, daily or occasional',
       'Diabetes', 'Exclusive breastfeeding, at least 6 months',
       'Has a regular healthcare provider', 'Heavy drinking',
       'High blood pressure',
       'Influenza immunization in the past 12 months',
       'Life satisfaction, satisfied or very satisfied', 'Mood disorder',
       'Perceived health, fair or poor',
       'Perceived health, very good or excellent',
       'Perceived life stress, most days quite a bit or extremely stressful',
       'Perceived mental health, fair or poor',
       'Perceived mental health, very good or excellent',
       'Physical activity, 150 minutes per week, adult (18 years and over)',
       'Physical activity, average 60 minutes per day, youth (12 to 17 years old)',
       'Sense of belonging to local community, somewhat strong or very strong']]

merged.info()
merged


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 78 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   HR_UID                                                                           16 non-null     int64  
 1   HSDA_Name                                                                        16 non-null     object 
 2   HSDPOP                                                                           16 non-null     float64
 3   peer_group                                                                       16 non-null     object 
 4   peer_char                                                                        16 non-null     object 
 5   HA_Name                                                                          16 non-null     object 
 6   Population d

,HR_UID,HSDA_Name,HSDPOP,peer_group,peer_char,HA_Name,Population density per square kilometre,15 to 64 years,65 years and over,Median age of the population,Average household size,Median total income of households in 2015 ($),Median after-tax income of households in 2015 ($),Single-detached house prop,Married or living common law prop,Not married and not living common law prop,Owner prop,Renter prop,One person or fewer per room prop,More than 1 person per room prop,Suitable prop,Not suitable prop,"No certificate, diploma or degree prop",Secondary (high) school diploma or equivalency certificate prop,"Postsecondary certificate, diploma or degree prop",In the labour force prop,Employed prop,Unemployed prop,Not in the labour force prop,"Car, truck, van - as a driver prop",Non-movers prop,Movers prop,amenity_dense,prox_idx_emp_HSDwt,prox_idx_pharma_HSDwt,prox_idx_childcare_HSDwt,prox_idx_health_HSDwt,prox_idx_grocery_HSDwt,prox_idx_educpri_HSDwt,prox_idx_educsec_HSDwt,prox_idx_lib_HSDwt,prox_idx_parks_HSDwt,prox_idx_transit_HSDwt,prox_idx_emp_HSDmed,prox_idx_pharma_HSDmed,prox_idx_childcare_HSDmed,prox_idx_health_HSDmed,prox_idx_grocery_HSDmed,prox_idx_educpri_HSDmed,prox_idx_educsec_HSDmed,prox_idx_lib_HSDmed,prox_idx_parks_HSDmed,prox_idx_transit_HSDmed,Arthritis (15 years and over),Asthma,"Body mass index, adjusted self-reported, adult (18 years and over), obese","Body mass index, adjusted self-reported, adult (18 years and over), overweight","Body mass index, self-reported, youth (12 to 17 years old), overweight or obese",Breast milk feeding initiation,Chronic obstructive pulmonary disease (COPD; 35 years and over),"Current smoker, daily","Current smoker, daily or occasional",Diabetes,"Exclusive breastfeeding, at least 6 months",Has a regular healthcare provider,Heavy drinking,High blood pressure,Influenza immunization in the past 12 months,"Life satisfaction, satisfied or very satisfied",Mood disorder,"Perceived health, fair or poor","Perceived health, very good or excellent","Perceived life stress, most days quite a bit or extremely stressful","Perceived mental health, fair or poor","Perceived mental health, very good or excellent","Physical activity, 150 minutes per week, adult (18 years and over)","Physical activity, average 60 minutes per day, youth (12 to 17 years old)","Sense of belonging to local community, somewhat strong or very strong"
0,5942,Central Vancouver Island,270229.0,C,Urban-rural mix,Vancouver Island,22.2,60.8,25.5,50.3,2.2,62238.0,55443.0,0.686457,0.602353,0.397625,0.747203,0.250212,0.992965,0.006993,0.971519,0.028481,0.168131,0.295599,0.536314,0.561081,0.921212,0.078828,0.438919,0.813119,0.848688,0.151331,0.058901,0.002829,0.003438,0.006786,0.001929,0.003677,0.010280,0.015565,0.005572,0.012972,0.000770,0.00450,0.02370,0.02040,0.0020,0.03990,0.05650,0.08460,0.0753,0.04470,0.0015,25.8,9.3,23.7,39.7,26.8,92.1,6.0,9.1,12.6,5.9,49.1,87.9,19.8,17.8,36.4,92.9,11.6,13.8,56.5,18.6,10.3,66.8,65.3,60.0,72.4
1,5911,East Kootenay,81058.0,D,Rural,Interior,1.8,63.5,21.0,46.4,2.3,71383.0,62283.0,0.704944,0.636808,0.363192,0.780615,0.218804,0.994051,0.005949,0.979106,0.020894,0.169656,0.295151,0.535193,0.626150,0.921088,0.078791,0.373926,0.790681,0.862341,0.137659,0.029567,0.000409,0.000961,0.000862,0.000545,0.001308,0.003625,0.002900,0.002574,0.002042,0.000138,0.00160,0.02570,0.01630,0.0028,0.04170,0.05105,0.05570,0.0765,0.03030,0.0049,24.1,7.8,30.4,35.9,NaN,100.0,4.8,11.9,17.3,8.5,NaN,82.9,24.4,18.6,31.2,91.6,8.0,14.5,61.8,17.3,8.6,67.9,71.9,73.3,76.5
2,5921,Fraser East,300097.0,B,Moderate urban,Fraser,26.1,63.9,18.0,41.2,2.7,69321.0,61172.0,0.521651,0.600962,0.399018,0.731092,0.266691,0.978300,0.021700,0.952722,0.047278,0.207510,0.339386,0.453146,0.633949,0.932884,0.067116,0.366051,0.844468,0.845311,0.154706,0.093064,0.002974,0.003076,0.004358,0.001183,0.002797,0.010082,0.004472,0.004084,0.006252,0.000617,0.01010,0.02390,0.03040,0.0037,0.03500,0.06830,0.05730,0.0745,0.03890,0.0027,20.6,10.6,28.1,39.5,17.1,95.1,6.1,9.1,

In [13]:
merged.columns.values



array(['HR_UID', 'HSDA_Name', 'HSDPOP', 'peer_group', 'peer_char',
       'HA_Name', 'Population density per square kilometre',
       '15 to 64 years', '65 years and over',
       'Median age of the population', 'Average household size',
       'Median total income of households in 2015 ($)',
       'Median after-tax income of households in 2015 ($)',
       'Single-detached house prop', 'Married or living common law prop',
       'Not married and not living common law prop', 'Owner prop',
       'Renter prop', 'One person or fewer per room prop',
       'More than 1 person per room prop', 'Suitable prop',
       'Not suitable prop', 'No certificate, diploma or degree prop',
       'Secondary (high) school diploma or equivalency certificate prop',
       'Postsecondary certificate, diploma or degree prop',
       'In the labour force prop', 'Employed prop', 'Unemployed prop',
       'Not in the labour force prop',
       'Car, truck, van - as a driver prop', 'Non-movers prop',
       

In [14]:
# Import COVID data
covid = pd.read_csv('cases_by_HSD_071620.csv')
covid['HSDA_Name'] = covid['GEO_NAME'].str.replace(r' Health Service Delivery Area$', '')
covid.HSDA_Name.replace({'North West': 'Northwest'}, inplace=True)
covid.info()
covid

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   GEO_NAME   16 non-null     object
 1   count      16 non-null     int64 
 2   HSDA_Name  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


,GEO_NAME,count,HSDA_Name
0,Central Vancouver Island Health Service Delivery Area,28,Central Vancouver Island
1,East Kootenay Health Service Delivery Area,18,East Kootenay
2,Fraser East Health Service Delivery Area,521,Fraser East
3,Fraser North Health Service Delivery Area,440,Fraser North
4,Fraser South Health Service Delivery Area,706,Fraser South
5,Kootenay-Boundary Health Service Delivery Area,13,Kootenay-Boundary
6,North Shore/Coast Garibaldi Health Service Delivery Area,265,North Shore/Coast Garibaldi
7,North Vancouver Island Health Service Delivery Area,59,North Vancouver Island
8,North West Health Service Delivery Area,14,Northwest
9,Northeast Health Service Delivery Area,15,Northeast


In [15]:
final = pd.merge(left = merged, right = covid, how = 'outer',
                 left_on = 'HSDA_Name', right_on = 'HSDA_Name')

convert = ['count']
for each in convert:
    final[each + '_prop'] = final[each]/final['HSDPOP']

final.info()
final

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 81 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   HR_UID                                                                           16 non-null     int64  
 1   HSDA_Name                                                                        16 non-null     object 
 2   HSDPOP                                                                           16 non-null     float64
 3   peer_group                                                                       16 non-null     object 
 4   peer_char                                                                        16 non-null     object 
 5   HA_Name                                                                          16 non-null     object 
 6   Population d

,HR_UID,HSDA_Name,HSDPOP,peer_group,peer_char,HA_Name,Population density per square kilometre,15 to 64 years,65 years and over,Median age of the population,Average household size,Median total income of households in 2015 ($),Median after-tax income of households in 2015 ($),Single-detached house prop,Married or living common law prop,Not married and not living common law prop,Owner prop,Renter prop,One person or fewer per room prop,More than 1 person per room prop,Suitable prop,Not suitable prop,"No certificate, diploma or degree prop",Secondary (high) school diploma or equivalency certificate prop,"Postsecondary certificate, diploma or degree prop",In the labour force prop,Employed prop,Unemployed prop,Not in the labour force prop,"Car, truck, van - as a driver prop",Non-movers prop,Movers prop,amenity_dense,prox_idx_emp_HSDwt,prox_idx_pharma_HSDwt,prox_idx_childcare_HSDwt,prox_idx_health_HSDwt,prox_idx_grocery_HSDwt,prox_idx_educpri_HSDwt,prox_idx_educsec_HSDwt,prox_idx_lib_HSDwt,prox_idx_parks_HSDwt,prox_idx_transit_HSDwt,prox_idx_emp_HSDmed,prox_idx_pharma_HSDmed,prox_idx_childcare_HSDmed,prox_idx_health_HSDmed,prox_idx_grocery_HSDmed,prox_idx_educpri_HSDmed,prox_idx_educsec_HSDmed,prox_idx_lib_HSDmed,prox_idx_parks_HSDmed,prox_idx_transit_HSDmed,Arthritis (15 years and over),Asthma,"Body mass index, adjusted self-reported, adult (18 years and over), obese","Body mass index, adjusted self-reported, adult (18 years and over), overweight","Body mass index, self-reported, youth (12 to 17 years old), overweight or obese",Breast milk feeding initiation,Chronic obstructive pulmonary disease (COPD; 35 years and over),"Current smoker, daily","Current smoker, daily or occasional",Diabetes,"Exclusive breastfeeding, at least 6 months",Has a regular healthcare provider,Heavy drinking,High blood pressure,Influenza immunization in the past 12 months,"Life satisfaction, satisfied or very satisfied",Mood disorder,"Perceived health, fair or poor","Perceived health, very good or excellent","Perceived life stress, most days quite a bit or extremely stressful","Perceived mental health, fair or poor","Perceived mental health, very good or excellent","Physical activity, 150 minutes per week, adult (18 years and over)","Physical activity, average 60 minutes per day, youth (12 to 17 years old)","Sense of belonging to local community, somewhat strong or very strong",GEO_NAME,count,count_prop
0,5942,Central Vancouver Island,270229.0,C,Urban-rural mix,Vancouver Island,22.2,60.8,25.5,50.3,2.2,62238.0,55443.0,0.686457,0.602353,0.397625,0.747203,0.250212,0.992965,0.006993,0.971519,0.028481,0.168131,0.295599,0.536314,0.561081,0.921212,0.078828,0.438919,0.813119,0.848688,0.151331,0.058901,0.002829,0.003438,0.006786,0.001929,0.003677,0.010280,0.015565,0.005572,0.012972,0.000770,0.00450,0.02370,0.02040,0.0020,0.03990,0.05650,0.08460,0.0753,0.04470,0.0015,25.8,9.3,23.7,39.7,26.8,92.1,6.0,9.1,12.6,5.9,49.1,87.9,19.8,17.8,36.4,92.9,11.6,13.8,56.5,18.6,10.3,66.8,65.3,60.0,72.4,Central Vancouver Island Health Service Delivery Area,28,0.000104
1,5911,East Kootenay,81058.0,D,Rural,Interior,1.8,63.5,21.0,46.4,2.3,71383.0,62283.0,0.704944,0.636808,0.363192,0.780615,0.218804,0.994051,0.005949,0.979106,0.020894,0.169656,0.295151,0.535193,0.626150,0.921088,0.078791,0.373926,0.790681,0.862341,0.137659,0.029567,0.000409,0.000961,0.000862,0.000545,0.001308,0.003625,0.002900,0.002574,0.002042,0.000138,0.00160,0.02570,0.01630,0.0028,0.04170,0.05105,0.05570,0.0765,0.03030,0.0049,24.1,7.8,30.4,35.9,NaN,100.0,4.8,11.9,17.3,8.5,NaN,82.9,24.4,18.6,31.2,91.6,8.0,14.5,61.8,17.3,8.6,67.9,71.9,73.3,76.5,East Kootenay Health Service Delivery Area,18,0.000222
2,5921,Fraser East,300097.0,B,Moderate urban,Fraser,26.1,63.9,18.0,41.2,2.7,69321.0,61172.0,0.521651,0.600962,0.399018,0.731092,0.266691,0.978300,0.021700,0.952722,0.047278,0.207510,0.339386,0.453146,0.633949,0.932884,0.067116,0.366051,0.844468,0.845311,0.154706,0.093064,0.002974,0.003076,0.004358,0.001183,0.002797,0.010082,0.00

In [16]:
# Export final df
final.to_csv('merged_data_BC_HSD.csv')